# GreenPower RAG System v05 - Content-Based Private Filter

## Pipeline complet:
1. Setup Qdrant (local vectoriel)
2. Chargement documents GreenPower via Gradio
3. Chunking + Embeddings (sentence-transformers)
4. Stockage Qdrant vector index
5. RAG pipeline : query → retrieval → **scan content for private_** → filter → Mistral
6. Interface Gradio interactive

## Nouveautés v05:
- ✨ **Filtrage par contenu**: Détecte les valeurs `private_*` DANS le texte des chunks
- ✨ **Analyse post-retrieval**: Scanne chaque chunk retourné par Qdrant
- ✨ **Message de confidentialité**: "Désolé, donnée confidentielle" si chunks contiennent private_
- ✨ **Pattern matching**: Regex pour détecter tout pattern `private_xxx`

## Comment ça marche:
- Un document normal peut contenir des références à des données privées
- Exemple: `"Le client private_client_001 a commandé..."`
- Le système détecte `private_client_001` et filtre ce chunk automatiquement
- L'utilisateur ne verra jamais ces informations sensibles

In [ ]:
# Cell 1: Installation des dépendances
!pip install -q langchain-mistralai langchain-community langchain-text-splitters
!pip install -q qdrant-client gradio sentence-transformers
!pip install -q pypdf python-docx python-dotenv

In [ ]:
# Cell 2: Imports
import os
import re
from pathlib import Path
import gradio as gr
from langchain_mistralai import ChatMistralAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
import uuid
from typing import List, Tuple
import pypdf
import docx
import json
import csv
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

print("✅ Imports OK")

In [ ]:
env_path = Path('.env')
if env_path.exists():
    with open(env_path) as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                key, value = line.split('=', 1)
                os.environ[key.strip()] = value.strip()

MISTRAL_API_KEY = os.getenv('MISTRAL_API_KEY')
QDRANT_URL = os.getenv('QDRANT_URL', ':memory:')  # Use :memory: for local or cloud URL
QDRANT_API_KEY = os.getenv('QDRANT_API_KEY', None)

# 🔒 PATTERN POUR DÉTECTER LES DONNÉES PRIVÉES
PRIVATE_PATTERN = re.compile(r'private_\w+', re.IGNORECASE)

In [ ]:
# Cell 4: Initialisation des clients

# Configuration CHUNKS
CHUNK_SIZE = 500
CHUNK_OVERLAP = 50
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"

# Initialize components
embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
llm = ChatMistralAI(model='mistral-small-latest', mistral_api_key=MISTRAL_API_KEY, temperature=0.7)

qdrant_client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY)
COLLECTION_NAME = "greenpower_docs"

# Text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    length_function=len,
)

print("✅ Clients initialisés")
print(f"   - Qdrant: Mode {QDRANT_URL}")
print(f"   - Embeddings: {EMBEDDING_MODEL}")
print(f"   - LLM: Mistral Small")
print(f"   - 🔒 Pattern privé: {PRIVATE_PATTERN.pattern}")

In [ ]:
# Cell 5: Création de la collection Qdrant

# Obtenir la dimension des embeddings en générant un embedding de test
test_embedding = embeddings.embed_query("test")
VECTOR_SIZE = len(test_embedding)

def create_collection_if_not_exists():
    """Crée la collection Qdrant si elle n'existe pas déjà"""
    try:
        collections = qdrant_client.get_collections()
        collection_names = [c.name for c in collections.collections]
        
        if COLLECTION_NAME in collection_names:
            print(f"ℹ️ Collection '{COLLECTION_NAME}' existe déjà")
            return
        
        qdrant_client.create_collection(
            collection_name=COLLECTION_NAME,
            vectors_config=VectorParams(
                size=VECTOR_SIZE,
                distance=Distance.COSINE
            )
        )
        print(f"✅ Collection '{COLLECTION_NAME}' créée (dimension: {VECTOR_SIZE})")
    except Exception as e:
        print(f"⚠️ Erreur création collection: {e}")
        raise

# Créer la collection
create_collection_if_not_exists()

In [ ]:
# Cell 6: Fonctions utilitaires

def extract_text_from_file(file_path: str) -> str:
    """Extrait le texte d'un fichier PDF, DOCX, TXT, JSON ou CSV"""
    file_path = Path(file_path)
    
    try:
        if file_path.suffix.lower() == '.pdf':
            with open(file_path, 'rb') as f:
                pdf_reader = pypdf.PdfReader(f)
                text = ""
                for page in pdf_reader.pages:
                    text += page.extract_text() + "\n"
                return text
        
        elif file_path.suffix.lower() in ['.docx', '.doc']:
            doc = docx.Document(file_path)
            return "\n".join([para.text for para in doc.paragraphs])
        
        elif file_path.suffix.lower() == '.txt':
            with open(file_path, 'r', encoding='utf-8') as f:
                return f.read()
        
        elif file_path.suffix.lower() == '.json':
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                if isinstance(data, dict):
                    text_parts = []
                    for key, value in data.items():
                        if isinstance(value, (dict, list)):
                            text_parts.append(f"{key}: {json.dumps(value, ensure_ascii=False, indent=2)}")
                        else:
                            text_parts.append(f"{key}: {value}")
                    return "\n".join(text_parts)
                elif isinstance(data, list):
                    return "\n\n".join([json.dumps(item, ensure_ascii=False, indent=2) for item in data])
                else:
                    return str(data)
        
        elif file_path.suffix.lower() == '.csv':
            with open(file_path, 'r', encoding='utf-8') as f:
                csv_reader = csv.DictReader(f)
                rows = []
                for row in csv_reader:
                    row_text = ", ".join([f"{key}: {value}" for key, value in row.items()])
                    rows.append(row_text)
                return "\n".join(rows)
        
        else:
            return f"Format non supporté: {file_path.suffix}"
    
    except Exception as e:
        return f"Erreur lecture fichier {file_path.name}: {str(e)}"

def contains_private_data(text: str) -> Tuple[bool, List[str]]:
    """🔒 Détecte si le texte contient des valeurs privées (private_xxx)
    
    Returns:
        Tuple[bool, List[str]]: (has_private, list_of_private_values)
    """
    matches = PRIVATE_PATTERN.findall(text)
    return len(matches) > 0, matches

def chunk_and_embed(text: str, source: str) -> List[dict]:
    """Découpe le texte en chunks et génère les embeddings"""
    # Chunking
    chunks = text_splitter.split_text(text)
    
    # Embeddings
    chunk_embeddings = embeddings.embed_documents(chunks)
    
    # Préparer les points pour Qdrant
    points = []
    for i, (chunk, embedding) in enumerate(zip(chunks, chunk_embeddings)):
        point = PointStruct(
            id=str(uuid.uuid4()),
            vector=embedding,
            payload={
                "text": chunk,
                "source": source,
                "chunk_index": i
            }
        )
        points.append(point)
    
    return points

print("✅ Fonctions utilitaires définies (avec détection private_ dans le contenu)")

In [ ]:
# Cell 7: Fonction d'upload de documents

def upload_documents(files):
    """Traite et stocke les documents uploadés"""
    if not files:
        return "❌ Aucun fichier uploadé"
    
    # S'assurer que la collection existe
    create_collection_if_not_exists()
    
    total_chunks = 0
    chunks_with_private_content = 0
    results = []
    
    for file in files:
        try:
            # Extraction du texte
            file_path = file.name if hasattr(file, 'name') else file
            filename = Path(file_path).name
            
            text = extract_text_from_file(file_path)
            
            if text.startswith("Erreur") or text.startswith("Format non supporté"):
                results.append(f"⚠️ {filename}: {text}")
                continue
            
            # Chunking et embedding
            points = chunk_and_embed(text, filename)
            
            # 🔍 Analyser combien de chunks contiennent des données privées
            private_chunks_in_doc = 0
            for point in points:
                has_private, _ = contains_private_data(point.payload["text"])
                if has_private:
                    private_chunks_in_doc += 1
            
            # Stockage dans Qdrant
            qdrant_client.upsert(
                collection_name=COLLECTION_NAME,
                points=points
            )
            
            total_chunks += len(points)
            chunks_with_private_content += private_chunks_in_doc
            
            if private_chunks_in_doc > 0:
                results.append(
                    f"✅ {filename}: {len(points)} chunks "
                    f"(⚠️ {private_chunks_in_doc} contiennent des données private_)"
                )
            else:
                results.append(f"✅ {filename}: {len(points)} chunks")
            
        except Exception as e:
            results.append(f"❌ {filename}: Erreur - {str(e)}")
    
    summary = f"\n\n📊 **Total: {total_chunks} chunks** stockés dans '{COLLECTION_NAME}'\n"
    if chunks_with_private_content > 0:
        summary += f"⚠️ **{chunks_with_private_content} chunks** contiennent des valeurs private_* "
        summary += "(seront filtrés lors des recherches)"
    
    return "\n".join(results) + summary

print("✅ Fonction upload_documents définie")

In [ ]:
# Cell 8: Fonction RAG avec filtrage des contenus privés

def search_and_answer(question: str, top_k: int = 3):
    """🔒 Recherche dans Qdrant et génère une réponse (filtre les chunks avec private_*)"""
    if not question or question.strip() == "":
        return "⚠️ Veuillez poser une question"
    
    try:
        # 1. Générer l'embedding de la question
        question_embedding = embeddings.embed_query(question)
        
        # 2. Recherche dans Qdrant avec un top_k plus large pour compenser le filtrage
        search_results = qdrant_client.search(
            collection_name=COLLECTION_NAME,
            query_vector=question_embedding,
            limit=top_k * 5  # Chercher 5x plus pour avoir assez après filtrage
        )
        
        if not search_results:
            return "⚠️ Aucun document trouvé. Uploadez d'abord des documents."
        
        # 3. 🔒 FILTRAGE DES CHUNKS CONTENANT DES DONNÉES PRIVÉES
        filtered_results = []
        private_chunks = []
        private_values_found = set()
        
        for point in search_results:
            chunk_text = point.payload["text"]
            
            # Vérifier si le chunk contient des valeurs private_
            has_private, private_matches = contains_private_data(chunk_text)
            
            if has_private:
                # 🚫 Ce chunk contient des données privées, on le stocke mais ne l'utilise pas
                private_chunks.append({
                    'source': point.payload.get('source', 'Unknown'),
                    'matches': private_matches
                })
                private_values_found.update(private_matches)
                continue
            
            # ✅ Chunk clean, on l'ajoute
            filtered_results.append(point)
            
            # Limiter au top_k demandé
            if len(filtered_results) >= top_k:
                break
        
        # 4. Vérifier s'il reste des résultats après filtrage
        if not filtered_results:
            # Tous les résultats contenaient des données privées
            return f"""## 🔒 Donnée confidentielle

Désolé, les informations pertinentes pour cette question contiennent des **données confidentielles** et ne peuvent pas être affichées.

---

🔒 **{len(private_chunks)} chunks** ont été trouvés mais contenaient des valeurs privées :
{', '.join(sorted(private_values_found))}

💡 Ces données sont protégées et ne seront jamais affichées dans les réponses.
"""
        
        # 5. Extraire les chunks pertinents (uniquement ceux sans données privées)
        contexts = []
        sources = []
        for point in filtered_results:
            contexts.append(point.payload["text"])
            source = point.payload.get("source", "Unknown")
            if source not in sources:
                sources.append(source)
        
        context_text = "\n\n---\n\n".join(contexts)
        
        # 6. Construire le prompt
        prompt = f"""Tu es un assistant qui répond aux questions en te basant UNIQUEMENT sur le contexte fourni.

Contexte:
{context_text}

Question: {question}

Instructions:
- Réponds de manière claire et concise
- Base-toi UNIQUEMENT sur le contexte fourni
- Si l'information n'est pas dans le contexte, dis-le clairement
- Cite les sources quand pertinent

Réponse:"""
        
        # 7. Générer la réponse avec Mistral
        response = llm.invoke(prompt)
        answer = response.content
        
        # 8. Formater la réponse avec info sur filtrage
        output = f"""## 💬 Réponse

{answer}

---

📚 **Sources consultées:** {', '.join(sources)}
🔍 **{len(contexts)} chunks** analysés (sans données confidentielles)
"""
        
        # Ajouter un warning si des chunks privés ont été exclus
        if private_chunks:
            output += f"\n\n🔒 **{len(private_chunks)} chunks** ont été exclus car ils contenaient des valeurs privées :"
            output += f"\n   {', '.join(sorted(private_values_found))}"
        
        return output
    
    except Exception as e:
        return f"❌ Erreur: {str(e)}\n\nDétails technique: {type(e).__name__}"

print("✅ Fonction search_and_answer définie (avec scan de contenu pour private_)")

In [ ]:
# Cell 9: Interface Gradio

with gr.Blocks(title="GreenPower RAG v05 - Content Filter") as demo:
    gr.Markdown(
        """
        # 🌱 GreenPower RAG System v05 - 🔒 Content-Based Private Filter
        
        **Retrieval-Augmented Generation** avec **filtrage intelligent des contenus privés**
        
        ### 📋 Workflow:
        1. **Upload** vos documents (PDF, DOCX, TXT, JSON, CSV)
        2. Le système indexe tout le contenu dans Qdrant
        3. **Ask** vos questions
        4. Le système **scanne les résultats** et filtre automatiquement les chunks contenant `private_xxx`
        
        ### 🔒 Protection des données:
        - Détection automatique de patterns `private_*` dans le **contenu des chunks**
        - Ex: `private_client_001`, `private_salary_data`, `private_contract_123`
        - Ces chunks ne seront **jamais affichés** même si pertinents pour la question
        
        ### 💡 Exemple:
        ```
        Document contient:
        "Notre objectif 2025 est de croître de 20%."
        "Le client private_client_001 représente 15% du CA."
        
        Question: "Quels sont les objectifs 2025?"
        → ✅ Répond avec le premier chunk
        → 🔒 Ignore le second (contient private_client_001)
        ```
        """
    )
    
    with gr.Tab("📤 Upload Documents"):
        gr.Markdown(
            """
            ### Upload vos documents
            
            Formats supportés: **PDF**, **DOCX**, **TXT**, **JSON**, **CSV**
            
            🔒 **Données privées dans le contenu:**
            - Utilisez le pattern `private_xxx` pour marquer des valeurs sensibles
            - Ex: `"Le projet private_project_alpha coûte 2M€"`
            - Ces références seront automatiquement détectées et filtrées
            
            Les documents seront:
            - Découpés en chunks de 500 caractères
            - Vectorisés avec sentence-transformers
            - Stockés dans Qdrant
            - Scannés pour détecter les valeurs private_*
            """
        )
        
        file_input = gr.File(
            file_count="multiple",
            label="📁 Sélectionnez vos documents",
            file_types=[".pdf", ".docx", ".doc", ".txt", ".json", ".csv"]
        )
        upload_btn = gr.Button("🚀 Upload et Traiter", variant="primary")
        upload_output = gr.Textbox(
            label="📊 Résultat",
            lines=10,
            placeholder="Les résultats d'upload apparaîtront ici..."
        )
        
        upload_btn.click(
            upload_documents,
            inputs=file_input,
            outputs=upload_output
        )
    
    with gr.Tab("💬 Ask Questions"):
        gr.Markdown(
            """
            ### Posez vos questions
            
            Le système va:
            1. 🔍 Rechercher les passages pertinents dans Qdrant
            2. 🔒 **Scanner chaque chunk** pour détecter des valeurs `private_*`
            3. 🚫 **Filtrer automatiquement** les chunks contenant des données privées
            4. 🧠 Générer une réponse avec Mistral (uniquement données publiques)
            5. 📚 Citer les sources et indiquer les données filtrées
            
            ⚠️ Si tous les chunks pertinents contiennent `private_*` → **"Désolé, donnée confidentielle"**
            """
        )
        
        question_input = gr.Textbox(
            label="❓ Votre question",
            placeholder="Ex: Quels sont les objectifs de GreenPower pour 2025?",
            lines=3
        )
        
        top_k_slider = gr.Slider(
            minimum=1,
            maximum=10,
            value=3,
            step=1,
            label="🎯 Nombre de chunks à récupérer",
            info="Plus de chunks = plus de contexte (mais plus lent)"
        )
        
        ask_btn = gr.Button("🤔 Obtenir la Réponse", variant="primary")
        
        answer_output = gr.Markdown(
            label="💡 Réponse",
            value="*La réponse apparaîtra ici...*"
        )
        
        ask_btn.click(
            search_and_answer,
            inputs=[question_input, top_k_slider],
            outputs=answer_output
        )
        
        gr.Examples(
            examples=[
                ["Quels sont les objectifs principaux de l'entreprise?", 3],
                ["Quelle est la stratégie de développement?", 5],
                ["Quels sont les projets en cours?", 3],
            ],
            inputs=[question_input, top_k_slider],
        )
    
    with gr.Tab("ℹ️ Info"):
        gr.Markdown(
            f"""
            ### 🔧 Configuration Technique
            
            - **Vector DB:** Qdrant ({QDRANT_URL})
            - **Embeddings:** {EMBEDDING_MODEL}
            - **LLM:** Mistral Small
            - **Collection:** {COLLECTION_NAME}
            - **Chunk size:** {CHUNK_SIZE} caractères
            - **Overlap:** {CHUNK_OVERLAP} caractères
            - **🔒 Pattern privé:** `{PRIVATE_PATTERN.pattern}`
            
            ### 🔒 Système de Confidentialité par Contenu
            
            **Comment ça marche:**
            1. Les documents sont indexés normalement dans Qdrant (pas de pré-filtrage)
            2. Lors d'une recherche, Qdrant retourne les chunks les plus pertinents
            3. **Chaque chunk est scanné** avec une regex : `/private_\\w+/`
            4. Si un chunk contient `private_xxx` → **il est rejeté automatiquement**
            5. Seuls les chunks "clean" sont envoyés à Mistral
            6. L'utilisateur est informé si des données ont été filtrées
            
            **Patterns détectés:**
            - `private_client_001` ✅
            - `Private_Salary_Data` ✅ (case insensitive)
            - `private_contract_alpha` ✅
            - `PRIVATE_PROJECT_X` ✅
            - `privatestuff` ❌ (pas de underscore)
            - `my_private_thing` ❌ (ne commence pas par private_)
            
            **Exemple d'utilisation:**
            ```
            Document: strategie.txt
            ---
            Chunk 1: "Nos objectifs 2025 sont de croître de 25%."
            → ✅ PUBLIC (pas de private_)
            
            Chunk 2: "Le client private_client_acme représente 30% du CA."
            → 🔒 PRIVÉ (détecté: private_client_acme)
            
            Chunk 3: "Nous lançons 3 nouveaux produits en Q2."
            → ✅ PUBLIC
            
            Question: "Quels sont les objectifs?"
            → Utilise Chunks 1 et 3
            → Ignore Chunk 2
            → Indique "1 chunk exclu (private_client_acme)"
            ```
            
            ### 📝 Notes Importantes
            
            - ⚡ Le filtrage se fait **après la recherche vectorielle**
            - 🎯 Le système cherche 5x plus de chunks pour compenser le filtrage
            - 📊 Les chunks privés sont comptés et listés dans la réponse
            - 🔐 Aucune donnée privée ne peut "fuiter" dans les réponses de Mistral
            - 💾 Mode in-memory ne persiste pas les données entre redémarrages
            
            ### ✨ Nouveautés v05
            
            - ✅ Regex pattern matching pour `private_*`
            - ✅ Scan post-retrieval de tous les chunks
            - ✅ Filtrage intelligent avant envoi à Mistral
            - ✅ Reporting transparent des valeurs privées détectées
            - ✅ Message de confidentialité si tous les résultats sont privés
            - ✅ Liste des valeurs privées trouvées (ex: private_client_001, private_salary_data)
            """
        )

# Lancer l'interface
print("\n" + "="*60)
print("🚀 Lancement de l'interface Gradio v05 - Content Filter...")
print("="*60)

demo.launch(
    server_name="127.0.0.1",
    server_port=7688,
    share=False,
    show_error=True
)